In [1]:
from hexathello import aiPlayers, autoPlayer, engine, jable, printing

import numpy as np

from os import path

# Hexathello can be played as a game, or played by AI to train
# We're unlikely to intialize a Hexathello Engine directly. Instead, use ``autoPlayer`` to setup a game
#   for AI to play

# -- Settings
game_size: int = 5
player_count: int = 2

# First, choose our AIs. We can create the dumbest AI Agent, always choosing randomly,
#   by making a GreedyHexAgent but setting `p_random` to 1
# Indexing begins with 0 as it should. As a result, the 'second' player is "Player 1"; use the latter notation

ai_0: aiPlayers.HexAgent = aiPlayers.GreedyHexAgent(
    size = game_size,
    player_count = player_count,
    p_random = 1.0,
    player_id = 0
)
    
ai_1: aiPlayers.HexAgent = aiPlayers.GreedyHexAgent(
    size = game_size,
    player_count = player_count,
    p_random = 1.0,
    player_id = 1
)
    

In [2]:
# Have them play eachother
literalHistory: jable.JyFrame = autoPlayer.runHexathello_withAgents(
    agents = [ai_0, ai_1],
    size = game_size,
    logging_level = 0
)

Early end, no move for any player
RESULT: 33 - 9; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [33, 9]}


In [3]:
# a literal history shows the state at turn from the board's point of view. However, to do machine learning,
#  we need to shift the encoding of state as if it were from player 0's point of view. Otherwise, we would need
#  a different network for each player index; 2, 3 or 6 times as many

povHistory: jable.JyFrame = autoPlayer.povHistory_from_literalHistory(
    literalHistory
)

printing.prettyprint( povHistory )

player_count size history_type                ai_id turn_index current_player                                        board_state                                     action_choices                                      player_action  scores winner
------------ ---- ------------ -------------------- ---------- -------------- -------------------------------------------------- -------------------------------------------------- -------------------------------------------------- ------- ------
           2    5          pov GreendomHexAgent_0-5          0              0  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]  0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]  0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [33, 9]      0
           2    5          pov       RandomHexAgent          1              1  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]  0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]  0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [9, 33]      1
           2    

In [4]:
# To save histories to disk, we encode the vectors as integers by taking the binary representation
#   of the 0-1 vectors:
#     [1,0,1] -> 1*4 + 0*2 + 1*1 = 5  
povHistory_encoded: jable.JyFrame = autoPlayer.history_asInt(
    povHistory
)

# Test we encode and decode appropriately
povHistory_decoded: jable.JyFrame = autoPlayer.history_fromInt(
    povHistory_encoded
)
_povHistory_i: dict[ str, any ] = {}
_povHistory_decoded_i: dict[ str, any ] = {}

assert len( povHistory ) == len( povHistory_decoded )

for i in range( len( povHistory ) ):
    _povHistory_i: dict = povHistory[i]
    _povHistory_decoded_i: dict = povHistory_decoded[i]

    if not len( _povHistory_i["board_state"] ) == len( _povHistory_decoded_i["board_state"] ):
        print("# Incongruity in len 'board_state' at row index={}".format(i))
        print( _povHistory_i )
        print( _povHistory_decoded_i )
        raise Exception("Len Incongruity")
    #

    if not np.all(
        _povHistory_i["board_state"] == _povHistory_decoded_i["board_state"]
    ):
        print("# Incongruity in 'board_state' at row index={}".format(i))
        print( _povHistory_i )
        print( _povHistory_decoded_i )
        raise Exception("Incongruity")
    #

    if not len( _povHistory_i["player_action"] ) == len( _povHistory_decoded_i["player_action"] ):
        print("# Incongruity in len 'player_action' at row index={}".format(i))
        print( _povHistory_i )
        print( _povHistory_decoded_i )
        raise Exception("Len Incongruity")
    #

    if not np.all(
        _povHistory_i["player_action"] == _povHistory_decoded_i["player_action"]
    ):
        print("# Incongruity in 'player_action' at row index={}".format(i))
        print( _povHistory_i )
        print( _povHistory_decoded_i )
        raise Exception("Incongruity")
    #    
#/for i in range( len( povHistory ) )

del _povHistory_decoded_i
del _povHistory_i
del povHistory_decoded

print("# Encode/decode success")

# Encode/decode success


In [ ]:
# in `quickstart_recording_data` We'll look at saving many games to disk to use for machine learning